In [15]:
import os
from os.path import dirname, abspath
import pandas as pd
import valentine as valentine
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import DistributionBased
import pprint

In [16]:
PATH = "C:/Users/fende/Desktop/Dati"
print(PATH)

C:/Users/fende/Desktop/Dati


Load data using pandas

In [17]:
df1 = pd.read_csv(PATH + "/bing_covid-19_data.csv",   sep = ",", low_memory = False)
print(df1.columns)
df2 = pd.read_csv(PATH + "/ecdc_cases.csv", sep = ",", low_memory = False)
print(df2.columns)

Index(['id', 'updated', 'confirmed', 'confirmed_change', 'deaths',
       'deaths_change', 'recovered', 'recovered_change', 'latitude',
       'longitude', 'iso2', 'iso3', 'country_region', 'admin_region_1',
       'iso_subdivision', 'admin_region_2', 'load_time'],
      dtype='object')
Index(['date_rep', 'day', 'month', 'year', 'cases', 'deaths',
       'countries_and_territories', 'geo_id', 'country_territory_code',
       'pop_data_2018', 'continent_exp', 'load_date', 'iso_country',
       'daterep'],
      dtype='object')


Keeping only the common values between the 2 datasets

In [18]:
# presi una tupla a caso per ogni stato della colonna iso3 per evitare di prendere i primi record di ogni stato ed effettuare un random sampling
df1 = df1.groupby(["iso3"]).sample(1)
df2 = df2.groupby(["country_territory_code"]).sample(1)

merged = pd.merge(df1, df2, how = "inner", left_on='iso3', right_on='country_territory_code')

merged.rename(columns={"deaths_x": "deaths"}, inplace=True)

#col_names_df2 = list(df2.columns)
#merged = merged.drop(col_names_df2, axis=1)
merged = merged[df1.columns]

merged.to_csv(PATH + "/filtered_bing_ecdc.csv", index = False)
#df2.to_csv(PATH + "/examples/data/filtered_policy_tracker.csv", index = False)

Instantiate matcher and run

In [19]:
%%time
matcher = DistributionBased(0.15, 0.15)
matches = valentine_match(merged, df2, matcher)

CPU times: total: 3.97 s
Wall time: 4.8 s


Ground truth for metrics calculation

In [20]:
ground_truth = [
                ("iso3", "country_territory_code"),
                ("iso2", "iso_country"),
               ]

Metrics calculation

In [21]:
metrics = valentine_metrics.all_metrics(matches, ground_truth)

pp = pprint.PrettyPrinter(indent=4)
print("Found the following matches:")
pp.pprint(matches)

print("\nAccording to the ground truth:")
pp.pprint(ground_truth)

print("\nThese are the scores of the matcher:")
pp.pprint(metrics)

Found the following matches:
{   (('table_1', 'confirmed'), ('table_2', 'cases')): 0.8852111887843906,
    (('table_1', 'confirmed'), ('table_2', 'day')): 0.9495950498610347,
    (('table_1', 'confirmed'), ('table_2', 'deaths')): 0.7901008511844057,
    (('table_1', 'confirmed'), ('table_2', 'month')): 0.858413606004478,
    (('table_1', 'confirmed_change'), ('table_2', 'cases')): 0.9542985637072338,
    (('table_1', 'confirmed_change'), ('table_2', 'day')): 0.9294689804737749,
    (('table_1', 'confirmed_change'), ('table_2', 'deaths')): 0.8537946071414095,
    (('table_1', 'confirmed_change'), ('table_2', 'month')): 0.8974773121496267,
    (('table_1', 'country_region'), ('table_2', 'countries_and_territories')): 0.990701856534625,
    (('table_1', 'deaths'), ('table_2', 'cases')): 0.8717203357768185,
    (('table_1', 'deaths'), ('table_2', 'day')): 0.9169688169850726,
    (('table_1', 'deaths'), ('table_2', 'deaths')): 0.7657990814131562,
    (('table_1', 'deaths'), ('table_2', 'mon